# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fea4f9af3d0>
├── 'a' --> tensor([[-0.2512, -0.3344, -0.2022],
│                   [-0.1942,  1.3035, -2.8154]])
└── 'x' --> <FastTreeValue 0x7fea4f9af310>
    └── 'c' --> tensor([[ 0.2692, -1.7537, -0.9247, -0.7544],
                        [ 1.8622, -0.1832,  0.2623, -0.7626],
                        [ 0.5406,  1.6350, -0.1108,  0.0697]])

In [4]:
t.a

tensor([[-0.2512, -0.3344, -0.2022],
        [-0.1942,  1.3035, -2.8154]])

In [5]:
%timeit t.a

60.2 ns ± 0.0976 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fea4f9af3d0>
├── 'a' --> tensor([[ 0.3164,  0.9590,  0.1034],
│                   [ 0.6986, -1.7566, -0.9084]])
└── 'x' --> <FastTreeValue 0x7fea4f9af310>
    └── 'c' --> tensor([[ 0.2692, -1.7537, -0.9247, -0.7544],
                        [ 1.8622, -0.1832,  0.2623, -0.7626],
                        [ 0.5406,  1.6350, -0.1108,  0.0697]])

In [7]:
%timeit t.a = new_value

58.6 ns ± 0.0526 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2512, -0.3344, -0.2022],
               [-0.1942,  1.3035, -2.8154]]),
    x: Batch(
           c: tensor([[ 0.2692, -1.7537, -0.9247, -0.7544],
                      [ 1.8622, -0.1832,  0.2623, -0.7626],
                      [ 0.5406,  1.6350, -0.1108,  0.0697]]),
       ),
)

In [10]:
b.a

tensor([[-0.2512, -0.3344, -0.2022],
        [-0.1942,  1.3035, -2.8154]])

In [11]:
%timeit b.a

54.3 ns ± 0.0481 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.4142, -0.3499, -0.3473],
               [-1.1094,  1.2601,  0.9810]]),
    x: Batch(
           c: tensor([[ 0.2692, -1.7537, -0.9247, -0.7544],
                      [ 1.8622, -0.1832,  0.2623, -0.7626],
                      [ 0.5406,  1.6350, -0.1108,  0.0697]]),
       ),
)

In [13]:
%timeit b.a = new_value

489 ns ± 0.198 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

850 ns ± 0.11 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 88.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 1.02 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 2.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe9a0608640>
├── 'a' --> tensor([[[-0.2512, -0.3344, -0.2022],
│                    [-0.1942,  1.3035, -2.8154]],
│           
│                   [[-0.2512, -0.3344, -0.2022],
│                    [-0.1942,  1.3035, -2.8154]],
│           
│                   [[-0.2512, -0.3344, -0.2022],
│                    [-0.1942,  1.3035, -2.8154]],
│           
│                   [[-0.2512, -0.3344, -0.2022],
│                    [-0.1942,  1.3035, -2.8154]],
│           
│                   [[-0.2512, -0.3344, -0.2022],
│                    [-0.1942,  1.3035, -2.8154]],
│           
│                   [[-0.2512, -0.3344, -0.2022],
│                    [-0.1942,  1.3035, -2.8154]],
│           
│                   [[-0.2512, -0.3344, -0.2022],
│                    [-0.1942,  1.3035, -2.8154]],
│           
│                   [[-0.2512, -0.3344, -0.2022],
│                    [-0.1942,  1.3035, -2.8154]]])
└── 'x' --> <FastTreeValue 0x7fe9a06084c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.1 µs ± 23.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe9a062bf40>
├── 'a' --> tensor([[-0.2512, -0.3344, -0.2022],
│                   [-0.1942,  1.3035, -2.8154],
│                   [-0.2512, -0.3344, -0.2022],
│                   [-0.1942,  1.3035, -2.8154],
│                   [-0.2512, -0.3344, -0.2022],
│                   [-0.1942,  1.3035, -2.8154],
│                   [-0.2512, -0.3344, -0.2022],
│                   [-0.1942,  1.3035, -2.8154],
│                   [-0.2512, -0.3344, -0.2022],
│                   [-0.1942,  1.3035, -2.8154],
│                   [-0.2512, -0.3344, -0.2022],
│                   [-0.1942,  1.3035, -2.8154],
│                   [-0.2512, -0.3344, -0.2022],
│                   [-0.1942,  1.3035, -2.8154],
│                   [-0.2512, -0.3344, -0.2022],
│                   [-0.1942,  1.3035, -2.8154]])
└── 'x' --> <FastTreeValue 0x7fea4f9b6670>
    └── 'c' --> tensor([[ 0.2692, -1.7537, -0.9247, -0.7544],
                        [ 1.8622, -0.1832,  0.2623, -0.7626],
                 

In [23]:
%timeit t_cat(trees)

29.6 µs ± 55.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.9 µs ± 264 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2692, -1.7537, -0.9247, -0.7544],
                       [ 1.8622, -0.1832,  0.2623, -0.7626],
                       [ 0.5406,  1.6350, -0.1108,  0.0697]],
              
                      [[ 0.2692, -1.7537, -0.9247, -0.7544],
                       [ 1.8622, -0.1832,  0.2623, -0.7626],
                       [ 0.5406,  1.6350, -0.1108,  0.0697]],
              
                      [[ 0.2692, -1.7537, -0.9247, -0.7544],
                       [ 1.8622, -0.1832,  0.2623, -0.7626],
                       [ 0.5406,  1.6350, -0.1108,  0.0697]],
              
                      [[ 0.2692, -1.7537, -0.9247, -0.7544],
                       [ 1.8622, -0.1832,  0.2623, -0.7626],
                       [ 0.5406,  1.6350, -0.1108,  0.0697]],
              
                      [[ 0.2692, -1.7537, -0.9247, -0.7544],
                       [ 1.8622, -0.1832,  0.2623, -0.7626],
                       [ 0.5406,  1.6350, -0.1108,  0.0697]],

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 101 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2692, -1.7537, -0.9247, -0.7544],
                      [ 1.8622, -0.1832,  0.2623, -0.7626],
                      [ 0.5406,  1.6350, -0.1108,  0.0697],
                      [ 0.2692, -1.7537, -0.9247, -0.7544],
                      [ 1.8622, -0.1832,  0.2623, -0.7626],
                      [ 0.5406,  1.6350, -0.1108,  0.0697],
                      [ 0.2692, -1.7537, -0.9247, -0.7544],
                      [ 1.8622, -0.1832,  0.2623, -0.7626],
                      [ 0.5406,  1.6350, -0.1108,  0.0697],
                      [ 0.2692, -1.7537, -0.9247, -0.7544],
                      [ 1.8622, -0.1832,  0.2623, -0.7626],
                      [ 0.5406,  1.6350, -0.1108,  0.0697],
                      [ 0.2692, -1.7537, -0.9247, -0.7544],
                      [ 1.8622, -0.1832,  0.2623, -0.7626],
                      [ 0.5406,  1.6350, -0.1108,  0.0697],
                      [ 0.2692, -1.7537, -0.9247, -0.7544],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 1.24 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
